## Day 20: A Regular Map

https://adventofcode.com/2018/day/20

### Part 1

I can't think of a way of generating the paths from the regex directly, so run through each possibility, drawing the map, and then do a breadth first search until the last room is found.

Do this a character at a time. The map is represented as a graph, with rooms as nodes. Maintain a set of processing nodes, the possible rooms reached at the current character. Branches are tracked with a stack.

If the next character is a direction that direction is applied to the processing nodes and the joining edges are added to the graph.

If the next character is an opening bracket then the path branches so add the processing nodes to the stack as the root of the branches, together with an empty set which will become the set of all leaves of the branch, all possible final rooms in the branches' paths.

Continue going through the rooms until the branch ends, either with a bar `|` or a closing bracket. The set of processing nodes is then added to the set of the branches' leaves. 

If the current character is a bar the branches's root becomes the processing nodes again and  then continues working through the new branch's directions.

Otherwise there's a closing bracket and these branches are dealt with. The top of the stack is removed and the processing nodes are set to the leaves of the branches. 

In [1]:
import networkx as nx


def next_room(current_room, direction):
    x, y = current_room
    
    if direction == 'N':
        return (x, y + 1)
    elif direction == 'S':
        return (x, y - 1)
    elif direction == 'W':
        return (x - 1, y)
    elif direction == 'E':
        return (x + 1, y)
    else:
        print(current_room, direction)
    
    
def draw_map(regex):
    facility = nx.Graph()
    processing = {(0,0)}
    stack = []
    
    for c in regex[1:-1]:
        if c == '(':
            stack.append([processing, set()])
        elif c in '|)':
            stack[-1][1] |= processing
            if c == '|':
                processing = stack[-1][0]
            elif c == ')':
                processing = stack[-1][1]
                stack.pop()
        else:
            next_rooms = {(this_room, next_room(this_room, c))
                          for this_room in processing}
            for a, b in next_rooms:
                facility.add_edge(a, b)
            processing = {x for _, x in next_rooms}
            
    return facility

That's shorter and less fiddly than I was expecting. Does it work?

In [2]:
test_facility = draw_map('^ENWWW(NEEE|SSE(EE|N))$')
test_facility.edges

EdgeView([((0, 0), (1, 0)), ((1, 0), (1, 1)), ((1, 1), (0, 1)), ((0, 1), (-1, 1)), ((-1, 1), (-2, 1)), ((-2, 1), (-2, 2)), ((-2, 1), (-2, 0)), ((-2, 2), (-1, 2)), ((-1, 2), (0, 2)), ((0, 2), (1, 2)), ((-2, 0), (-2, -1)), ((-2, -1), (-1, -1)), ((-1, -1), (0, -1)), ((-1, -1), (-1, 0)), ((0, -1), (1, -1))])

It appears so.

There's probably something in NetworkX that will find the furthest shortest path but the algorithm is simple enough so hand knit it. This is a form of BFS, where at each increasing depth of search all of the nodes adjacent to those at the previous depth are added. The set of nodes discovered in the last iteration together with their depth are returned.

In [3]:
def furthest_rooms(facility):
    seen = set([(0,0)])
    last_seen = set([(0,0)])
    n = 0

    while True:
        next_seen = set().union(*[list(facility.neighbors(r)) 
                                  for r in last_seen]) - seen
        if not next_seen: 
            return n, last_seen
        
        n += 1
        seen |= next_seen
        last_seen = next_seen

In [4]:
furthest_rooms(test_facility)

(10, {(1, -1)})

That's the right answer for the first example, let's try the rest.

In [5]:
test_cases = [
    '^WNE$',
    '^ENWWW(NEEE|SSE(EE|N))$',
    '^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$',
    '^ESSWWN(E|NNENN(EESS(WNSE|)SSS|WWWSSSSE(SW|NNNE)))$',
    '^WSSEESWWWNW(S|NENNEEEENN(ESSSSW(NWSW|SSEN)|WSWWN(E|WWS(E|SS))))$'
]

for re in test_cases:
    print(furthest_rooms(draw_map(re)))

(3, {(0, 1)})
(10, {(1, -1)})
(18, {(2, 2)})
(23, {(-1, 2)})
(31, {(0, -1), (-3, 0), (3, -2)})


Splendid. Now for the rather lengthy input.

In [6]:
%time facility = draw_map(open('input', 'r').read().strip())
%time furthest_rooms(facility)

CPU times: user 129 ms, sys: 8.04 ms, total: 137 ms
Wall time: 136 ms
CPU times: user 25.9 ms, sys: 0 ns, total: 25.9 ms
Wall time: 25.8 ms


(3151, {(-30, 5), (-28, 7)})

Blimey, right first time and fairly nippy with it. It turns out that going off and thinking about these problems and writing the algorithm down before implementing it makes the code much easier. Who knew? (I had a similar experience with [Day 17](https://github.com/mratford/advent/blob/master/2018/17/Reservoir%20Research.ipynb).)

### Part 2

A small change to the search function will suffice.

In [7]:
def thousand_doors(facility):
    seen = set([(0,0)])
    last_seen = set([(0,0)])
    n = 0
    answer = 0

    while True:
        next_seen = set().union(*[list(facility.neighbors(r)) 
                                  for r in last_seen]) - seen
        if not next_seen: 
            return answer
        
        n += 1
        seen |= next_seen
        last_seen = next_seen
        if n >= 1000:
            answer += len(next_seen)

In [8]:
thousand_doors(facility)

8784

I'm five days behind so Merry Christmas!